## Adding new song data

Run the following to update the dataframe with new music of necessary (since the HITS Streaming Songs worksheets updates weekly on Friday). This Jupyter Notebook follows a similar structure to part 1a of this project

In [1]:
#to access spotipy
import configparser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

#to allow interaction with underlying operating system
import os

#to parse through spotipy's output
import json

#for dataframe cleaning
import numpy as np
import pandas as pd

#to access google sheets with python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#to edit google sheets from python
from df2gspread import df2gspread as d2g

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

#import spotify client credentials 

##  CONVERT THE FOLLOWING TO CODE

#The following creates environmental variables

%env SPOTIPY_CLIENT_ID=YOUR_SPOTIFY_APP_CLIENT_ID

%env SPOTIPY_CLIENT_SECRET=YOUR_SPOTIFY_APP_CLIENT_SECRET_ID

%env SPOTIPY_REDIRECT_URI=http://127.0.0.1:8080/
        

#output is hidden/cell is converted to "markdown" to protect privacy

In [3]:
#import spotify client credentials 
spotify_client_id = os.environ['SPOTIPY_CLIENT_ID']
spotify_secret = os.environ['SPOTIPY_CLIENT_SECRET']
spotify_redirect_uri = os.environ['SPOTIPY_REDIRECT_URI']

#create a Spotify instance
client_credentials_manager = SpotifyClientCredentials(client_id=spotify_client_id, client_secret=spotify_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#to confirm 
print(sp)

In [4]:
#service account
sa = gspread.service_account("gspread_service_account.json")

#sheet
sh = sa.open("HITS Streaming Songs")

#worksheet
wks = sh.worksheet("HITS Streaming Songs")

#uses all values in the worksheet for the data frame
gsheets_df_updated = pd.DataFrame(wks.get_all_values())

header = gsheets_df_updated.iloc[0] #isolate first row as header
gsheets_df_updated = gsheets_df_updated[1:] #get rid of header in original df
gsheets_df_updated.columns = header

gsheets_df_updated = gsheets_df_updated.set_index('index') #set 'index' as index

gsheets_df_updated['title'] = gsheets_df_updated['title'].str.rstrip(' ')
gsheets_df_updated['artist'] = gsheets_df_updated['artist'].str.rstrip(' ')
gsheets_df_updated['album'] = gsheets_df_updated['album'].str.rstrip(' ')

#convert index from string to int
gsheets_df_updated.index = gsheets_df_updated.index.astype(int)

#fill in blank values with 0
gsheets_df_updated = gsheets_df_updated.replace(r'^\s*$', "0", regex=True)

#convert all numbers-as-strings to numbers from row 4 in python's index and onwards
for col_name in gsheets_df_updated.columns.to_list()[4:len(gsheets_df_updated.columns.to_list())]:
    
    #get rid of strings' commas and convert strings to integers
    gsheets_df_updated[col_name] = gsheets_df_updated[col_name].str.replace(',','').astype('int')
    
gsheets_df_updated

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,2022-06-10,...,2017-04-07,2017-03-31,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music
index,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09,5528407,0,5528407,0,0,0,...,5528407,5420007,0,0,0,0,0,10948414,36856,0
2,Fake Love,Drake,More Life,2016-11-04,31271140,0,7800314,0,0,0,...,7800314,8892358,0,0,0,0,0,40163498,52002,0
3,Starboy,The Weeknd,Starboy,2016-11-25,5225061,0,5225061,0,0,0,...,5225061,5329562,0,0,0,0,0,10554623,34834,0
4,Reminder,The Weeknd,Starboy,2016-11-25,5229932,0,5229932,0,0,0,...,0,0,0,0,0,0,0,5229932,34866,0
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,2016-12-16,23375700,0,6072680,0,0,0,...,0,0,0,0,0,0,0,23375700,40485,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,Down Hill,Drake,"Honestly, Nevermind",2022-06-17,10140085,10140085,10140085,10140085,10140085,0,...,0,0,0,0,0,0,0,0,0,0
2149,The Kind of Love We Make,Luke Combs,Growin' Up,2022-06-17,9097056,9097056,9097056,9097056,9097056,0,...,0,0,0,0,0,0,0,0,0,0
2150,Tie That Binds,Drake,"Honestly, Nevermind",2022-06-17,8688133,8688133,8688133,8688133,8688133,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#dg to gspread worksheet
df2gspread_wks = sh.worksheet("DF to Gspread")

#uses all values in the worksheet for the data frame
df2gspread_wks = pd.DataFrame(df2gspread_wks.get_all_values())

header = df2gspread_wks.iloc[0] #isolate first row as header
df2gspread_wks = df2gspread_wks[1:] #get rid of header in original df
df2gspread_wks.columns = header

df2gspread_wks = df2gspread_wks.set_index('index') #set 'index' as index

#convert index from string to int
df2gspread_wks.index = df2gspread_wks.index.astype(int)

#convert all numbers-as-strings to numbers
df2gspread_wks.loc[:,'streams_2017_to_present':'units'] = df2gspread_wks.loc[:,'streams_2017_to_present':'units'].astype(int)
df2gspread_wks

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,2022-06-10,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit
index,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09 00:00:00,5528407,0,5528407,0,0,0,...,0.0,8,0.111,-5.599,1,0.0338,95.01,4,0.661,False
2,Fake Love,Drake,More Life,2016-11-04 00:00:00,31271140,0,7800314,0,0,0,...,0.0,9,0.176,-9.35,0,0.287,134.007,4,0.613,True
3,Starboy,The Weeknd,Starboy,2016-11-25 00:00:00,5225061,0,5225061,0,0,0,...,6.35e-06,7,0.137,-7.015,1,0.276,186.003,4,0.486,True
4,Reminder,The Weeknd,Starboy,2016-11-25 00:00:00,5229932,0,5229932,0,0,0,...,0.0,8,0.164,-6.923,1,0.193,160.053,4,0.388,True
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,2016-12-16 00:00:00,23375700,0,6072680,0,0,0,...,0.000118,7,0.0707,-7.509,0,0.225,139.976,4,0.344,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,Down Hill,Drake,"Honestly, Nevermind",2022-06-17 00:00:00,10140085,10140085,10140085,10140085,10140085,0,...,0.00036,3,0.0895,-11.261,1,0.0435,109.994,4,0.639,False
2149,The Kind of Love We Make,Luke Combs,Growin' Up,2022-06-17 00:00:00,9097056,9097056,9097056,9097056,9097056,0,...,6.47e-06,1,0.0942,-4.13,1,0.0277,102.025,4,0.464,False
2150,Tie That Binds,Drake,"Honestly, Nevermind",2022-06-17 00:00:00,8688133,8688133,8688133,8688133,8688133,0,...,0.0917,4,0.108,-10.912,0,0.0593,120.017,4,0.284,False


In [6]:
final_df_updated = pd.concat([gsheets_df_updated, df2gspread_wks.loc[:,'uri':] ],
                            axis=1)

final_df_updated = final_df_updated.replace(np.nan, "")
final_df_updated.index = final_df_updated.index.astype(int)
final_df_updated

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,2022-06-10,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit
index,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09,5528407,0,5528407,0,0,0,...,0.0,8,0.111,-5.599,1,0.0338,95.01,4,0.661,False
2,Fake Love,Drake,More Life,2016-11-04,31271140,0,7800314,0,0,0,...,0.0,9,0.176,-9.35,0,0.287,134.007,4,0.613,True
3,Starboy,The Weeknd,Starboy,2016-11-25,5225061,0,5225061,0,0,0,...,6.35e-06,7,0.137,-7.015,1,0.276,186.003,4,0.486,True
4,Reminder,The Weeknd,Starboy,2016-11-25,5229932,0,5229932,0,0,0,...,0.0,8,0.164,-6.923,1,0.193,160.053,4,0.388,True
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,2016-12-16,23375700,0,6072680,0,0,0,...,0.000118,7,0.0707,-7.509,0,0.225,139.976,4,0.344,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,Down Hill,Drake,"Honestly, Nevermind",2022-06-17,10140085,10140085,10140085,10140085,10140085,0,...,0.00036,3,0.0895,-11.261,1,0.0435,109.994,4,0.639,False
2149,The Kind of Love We Make,Luke Combs,Growin' Up,2022-06-17,9097056,9097056,9097056,9097056,9097056,0,...,6.47e-06,1,0.0942,-4.13,1,0.0277,102.025,4,0.464,False
2150,Tie That Binds,Drake,"Honestly, Nevermind",2022-06-17,8688133,8688133,8688133,8688133,8688133,0,...,0.0917,4,0.108,-10.912,0,0.0593,120.017,4,0.284,False


In [7]:
def search_for_URI(data):
    """
    Returns track URIs and the explicit status for a song

        Parameters:
            data (series): series containing a song title, artist, and the song's album
        
        Returns:
            song_info_list (list): list of song title, artist, album title, song URI and explicit status
                #otherwise, returns just the song title, artist, and album title
    """
    song_to_search = data['title'].replace("’","").replace("'","").lower()
    album_to_search = data['album'].replace("’","").replace("'","").lower()
    to_search = song_to_search + " " + data['artist']
    
    #this function searches for q on Spotify
    result = sp.search(q=to_search)
        
    #loops through all results from search
    for song_info_index in range(len(result['tracks']['items'])): 

        #checks for results' album to match the input data's album title (taking into account apostrophe formats)
        if result['tracks']['items'][song_info_index]['album']['name'].replace("’","").replace("'","").lower() == album_to_search:
                        
            #checks for results' song title to match input data's song title (taking into account apostrophe formats)
            if result['tracks']['items'][song_info_index]['name'].replace("’","").replace("'","").lower() == song_to_search:     
                
                #empty list to be filled with name/artist/album/explicit/uri info
                song_info_list = []

                #checks if a song is explicit       
                if result['tracks']['items'][song_info_index]['explicit'] == True:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
                
                #first, if a song is NOT explicit
                if result['tracks']['items'][song_info_index]['explicit'] == False:
                    song_info_list.append(result['tracks']['items'][song_info_index]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['artists'][0]['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['album']['name'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['explicit'])
                    song_info_list.append(result['tracks']['items'][song_info_index]['uri'])
                    return song_info_list
    
    #return just the song title/artist/album title if the URI is not found
    return [data['title'],data['artist'],data['album'],"",""]

In [8]:
new_uri_list = []

[new_uri_list.append(search_for_URI(final_df_updated.loc[n+1,'title':'album'])) 
for n in range(len(final_df_updated.loc[:])) if final_df_updated.iloc[n]['uri'] == ""]

#convert list to DataFrame
new_uri_df = pd.DataFrame(new_uri_list,columns=["song","artist","album","explicit","uri"])

#change index to fit the updated Google Sheet's index
new_uri_df.index = range(len(final_df_updated)-len(new_uri_df)+1,len(final_df_updated)+1)

#add name for index column
new_uri_df.index.name = 'index'

In [9]:
#create a dataframe with all songs that have no URI
data_to_clean = new_uri_df[new_uri_df['uri'] == ""]
data_to_clean.index = range(1,len(data_to_clean)+1)

In [10]:
def song_uri_search(q):
    """
    Prints out different URIs (and other info) from a resulting search on spotipy
    
    Parameter:
        q (str): a string to look up using spotipy's search function
    """
    result = sp.search(q) 
    
    for song_info_dict in range(len(result['tracks']['items'])):
        print("Song: ", result['tracks']['items'][song_info_dict]['name'])
        print("Artist: ", result['tracks']['items'][song_info_dict]['artists'][0]['name'])
        print("Album: ", result['tracks']['items'][song_info_dict]['album']['name'])
        print("Explicit status: ", result['tracks']['items'][song_info_dict]['explicit'])
        print("URI: ", result['tracks']['items'][song_info_dict]['uri'],"\n")

In [11]:
def data_cleaning_func(data_to_clean, uri_df):
    #loop through every song in data_to_clean
    for i in range(0,len(data_to_clean)):
        print("Please follow the following steps to fill in the missing URI(s)." + "\n")
        song_to_search = data_to_clean.iloc[i]['song'].replace("’","").replace("'","")
        album_to_search = data_to_clean.iloc[i]['album'].replace("’","").replace("'","")
        artist_to_search = data_to_clean.iloc[i]['artist']
            
        #concatenate the song title, artist name, and album title to search
        to_search = song_to_search + " " + artist_to_search + " " + album_to_search

        result = sp.search(to_search)
        print("The following results are the search results from searching: '" + to_search + "'")
        print("\n")
            
        #call on song_uri_search function to print out search results
        song_uri_search(to_search)
        print("Search the URI(s) on your browser to double check the song is correct")
            
        #ensure user obtains desired result from the above search
        print("Did you obtain a desired result? Please type 'yes' or 'no': ")
        desired_result = str(input())
            
        #ensure desired_result is a valid input
        while (desired_result != 'yes')  and (desired_result != 'no'):
            print("Please give a valid input. Type either 'yes' or 'no': ")
            desired_result = str(input())
            
        #run the following in case the desired output is not available in the search above
        while desired_result == 'no':
            print("\nPlease manually search the information for this song (be as specific as possible): ")
            manual_search = str(input())
            song_uri_search(manual_search)
                
            #ask if another search is necessary
            print("\nWould you like to make another search? Please type in 'yes' or 'no': ")
            another_search = str(input())
                
            #ensure request for another search is valid
            while (another_search != 'yes')  and (another_search != 'no'):
                print("\nPlease give a valid input. Type either 'yes' or 'no': ")
                another_search = str(input())
                
            if another_search == "no":
                desired_result = 'yes'
            if another_search == "yes":
                desired_result = 'no'
            
        #the index of the song being searched, obtained from uri_df using song and album titles from data_to_clean
        missing_song_index = uri_df[(uri_df['song']==data_to_clean.iloc[i]['song']) & (uri_df['album']==data_to_clean.iloc[i]['album'])].index.values[0]
            
        #obtain explicit status of the song. Have user input the status because numerous songs can result from the search
        print("\nPlease copy and paste the explicit status of your selected song.")
        print("Alternatively, type in 'True' or 'False' (make sure the first character is capitalized): ")
        explicit_status = str(input())
            
        #check if explicit_status is valid
        while (explicit_status != "True")  and (explicit_status != "False"):
            print("\nPlease give a valid input. Type either 'True' or 'False': ")
            explicit_status = str(input())
            
        #to convert the string to a boolean
        explicit_status = eval(explicit_status)
            
        #obtain URI of the song. Have user input the URI because numerous songs can result from the search
        print("\nPlease copy and paste the entire URI for your selected song here: ")
        uri = str(input())
            
        #to give the user a chance to re-enter the URI in case they mispelled it
        print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
        confirmation = str(input())
            
        #check if confirmation is valid
        while (confirmation != 'yes')  and (confirmation != 'no'):
            print("\nPlease give a valid input. Type either 'yes' or 'no': ")
            confirmation = str(input())
            
        #ask for user to input URI again if they do not confirm they want to move foward. Continue asking after every input for uri
        if confirmation == 'no':
            print("\nPlease retype your URI here: ")
            uri = str(input())
                
            print("\nDouble check that your URI is correct. Ready to confirm? Please enter 'yes' or 'no': ")
            confirmation = str(input())
                
        #edit the uri dataframe!
        uri_df.loc[missing_song_index]['explicit'] = explicit_status
        uri_df.loc[missing_song_index]['uri'] = uri
            
        print("\nThe URI/explicit status for " + data_to_clean.loc[i]['song'] + " by " + artist_to_search + " is now updated on the uri_df!")
            
    #create a dataframe with all songs that have no URI
    data_to_clean = uri_df[uri_df['uri'] == ""]
    data_to_clean.reset_index(drop=True, inplace=True)
    
    print("\nAll URI/explicit statuses are accounted for!")

In [12]:
if len(data_to_clean) > 0:
    data_cleaning_func(data_to_clean, uri_df)

In [13]:
print("Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.")

Success! The entire dataframe is now updated with song URIs! Please wait for Spotify's track features to be retreived.


In [14]:
def getTrackFeatures(track_id):
    """
    Returns a list of selected information from Spotify's API given a song's URI

    Parameters:
        track_id (str): a track's URI
    
    Returns
        track (list): a list of song information including it's danceability, length, tempo, etc. 
    """
    md = sp.track(track_id)
    features = sp.audio_features(track_id)
    
    #meta data
    name = md['name']
    album = md['album']['name']
    artist = md['album']['artists'][0]['name']
    release_date = md['album']['release_date']
    length = md['duration_ms']
    popularity = md['popularity']
    
    #features from the data
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    mode = features[0]['mode']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    valence = features[0]['valence']
    
    #putting it all together
    track = [name, album, artist, release_date, length, popularity, 
             acousticness, danceability, energy, instrumentalness, key,
            liveness, loudness, mode, speechiness, 
             tempo, time_signature, valence]
    return track

In [15]:
new_track_features_list = []

[new_track_features_list.append(getTrackFeatures(new_uri_df['uri'][i]) ) for i in new_uri_df.index]

new_track_features_list

[]

In [16]:
new_api_info_df = pd.DataFrame(new_track_features_list ,columns=['name','album','artist',
    'release_date','length','popularity','acousticness','danceability',
    'energy','instrumentalness','key','liveness','loudness','mode',
    'speechiness','tempo','time_signature','valence'])

#change index to fit the updated Google Sheet's index
new_api_info_df.index = range(len(final_df_updated)-len(new_api_info_df)+1,len(final_df_updated)+1)

#change release_date to date format
new_api_info_df['release_date'] = pd.to_datetime(new_api_info_df['release_date'])

new_api_info_df

,name,album,artist,release_date,length,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [17]:
#concatinate all new variables into a final dataframe titled new_variables_to_add
new_variables_to_add = pd.concat(
    
    #this gets all the new song streaming data from the Google Sheets
    [final_df_updated.loc[len(final_df_updated)-len(new_api_info_df)+1:len(final_df_updated)+1,:'units'],
    new_uri_df.loc[:,'uri'],
    new_api_info_df.loc[:,'release_date':'valence'],
    new_uri_df.loc[:,'explicit']
    ],
    axis=1)
new_variables_to_add

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,2022-06-10,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit


In [18]:
#add variables_to_add to the end of the final_df_updated
df_to_save = pd.concat([
    final_df_updated.loc[:len(final_df_updated)-len(new_api_info_df),:],
    new_variables_to_add
])

#add name for index column
df_to_save.index.name = 'index'

#reset index and convert all integers/float/dates to strings in preparation to convert to a Google Shet
df_to_save = df_to_save.reset_index()
df_to_save

,index,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-06-17,...,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,explicit
0,1,Closer,The Chainsmokers,Closer,2016-09-09,5528407,0,5528407,0,0,...,0.0,8,0.111,-5.599,1,0.0338,95.01,4,0.661,False
1,2,Fake Love,Drake,More Life,2016-11-04,31271140,0,7800314,0,0,...,0.0,9,0.176,-9.35,0,0.287,134.007,4,0.613,True
2,3,Starboy,The Weeknd,Starboy,2016-11-25,5225061,0,5225061,0,0,...,6.35e-06,7,0.137,-7.015,1,0.276,186.003,4,0.486,True
3,4,Reminder,The Weeknd,Starboy,2016-11-25,5229932,0,5229932,0,0,...,0.0,8,0.164,-6.923,1,0.193,160.053,4,0.388,True
4,5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,2016-12-16,23375700,0,6072680,0,0,...,0.000118,7,0.0707,-7.509,0,0.225,139.976,4,0.344,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,2148,Down Hill,Drake,"Honestly, Nevermind",2022-06-17,10140085,10140085,10140085,10140085,10140085,...,0.00036,3,0.0895,-11.261,1,0.0435,109.994,4,0.639,False
2148,2149,The Kind of Love We Make,Luke Combs,Growin' Up,2022-06-17,9097056,9097056,9097056,9097056,9097056,...,6.47e-06,1,0.0942,-4.13,1,0.0277,102.025,4,0.464,False
2149,2150,Tie That Binds,Drake,"Honestly, Nevermind",2022-06-17,8688133,8688133,8688133,8688133,8688133,...,0.0917,4,0.108,-10.912,0,0.0593,120.017,4,0.284,False
2150,2151,U-Digg (feat. 42 Dugg & Veeze),Lil Baby,U-Digg (feat. 42 Dugg & Veeze),2022-06-17,8150009,8150009,8150009,8150009,8150009,...,0.0,1,0.266,-6.537,1,0.147,147.496,4,0.154,True


In [19]:
url = "https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0"

spreadsheet_key = url.split("/")[-2]

wks_name = 'DF to Gspread'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)

gc = gspread.authorize(credentials)

d2g.upload(df_to_save, spreadsheet_key, wks_name, credentials = credentials,
          row_names=False)

<Worksheet 'DF to Gspread' id:2071085388>

You can now use the worksheet above for analytical purposes!! Happy analyzing! :)